In [ ]:
# Импорт библиотек
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

## Информация по Датасету

In [ ]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
df.duplicated().sum()

54

In [ ]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**
***
1. **Пропущенные значения:**  
*days_employed*, *total_income*  
Эти данные отображают занятость клиента, которая прямо влияет на плетежеспособность.  
Предположительно, пропуски не случайны, возможно клиент специально скрыл эту информацию, надеясь на одобрение кредита.

2. **Проблемы в данных:**  
*days_employed*, *children* - встречаются отрицательтные значения, что не применимо к понятию стаж и количество детей.   
*education* - значения в разных регистрах.

3. **Дубликаты:**  
Есть дубликаты. Так как в данных нет разреза по дате/времени, дубликаты могут обозначать одного и того же человека с обращением в разное время. Следовательно их удаление ни как не повлияет на результаты исследований.


## Предобработка данных

In [ ]:
# переведем данные в нижний регистр и выполним проверку
df['education'] = df['education'].str.lower() 
df['education'].unique() 

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [ ]:
# переведем данные в положительные значения и выполним проверку
df['days_employed'] = df['days_employed'].abs() 
(df['days_employed'] <0).sum()

0

In [ ]:
df['children'] = df['children'].abs()
(df['children'] <0).sum()

0

In [ ]:
# Удалим дубликаты
df = df.drop_duplicates().reset_index(drop = True) 
df.duplicated().sum()

0

Изучим поля с пропусками, оценим возможность их замены.

In [ ]:
df_isna = df[df['days_employed'].isna()]
df_isna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2103 entries, 12 to 21439
Data columns (total 12 columns):
children            2103 non-null int64
days_employed       0 non-null float64
dob_years           2103 non-null int64
education           2103 non-null object
education_id        2103 non-null int64
family_status       2103 non-null object
family_status_id    2103 non-null int64
gender              2103 non-null object
income_type         2103 non-null object
debt                2103 non-null int64
total_income        0 non-null float64
purpose             2103 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 213.6+ KB


**Вывод** 
***
Доля значений с пропусками: 10% - это значительный объем.  
Рассматриваем 2 варианта:

1. **Проводим исследование без пропусков.**   
Аргументы:  
*Ежемесячный доход* - количественная переменная, которая необходима для решения задачи.  
Можем заменить ее на медиану, в зависимости от параметра "тип занятости", но расчеты дадут пошгрешность в результатах.


2. **Проводим исследование с заполнением пропусков.**    
Аргументы:  
Из-за удаления данных будет смещение по номинативной переменной, что может значитльно повлиять на выводы в ииследованиях.

In [ ]:
# Сохраним DataFrame без пропусков в отдельную переменную для исследования по 1 варианту
df_notna = df.dropna().reset_index(drop = True)
df_notna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19351 entries, 0 to 19350
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null float64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null float64
purpose             19351 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 1.8+ MB


Проведем заполнение пропусков для проведения исследования по 2 варианту  
Чтобы исключить влияние выбросов, заполнять пропуски будем медианой.

In [ ]:
# Медиана по типу занятости
median_income = df_notna.groupby('income_type')['total_income'].agg(['median']).sort_values(by='median', ascending=False)
median_income

,median
income_type,
предприниматель,499163.144947
компаньон,172357.950966
госслужащий,150447.935283
сотрудник,142594.396847
безработный,131339.751676
пенсионер,118514.486412
студент,98201.625314
в декрете,53829.130729


In [ ]:
# Медиана по стажу работы
median_days_employed = df_notna['days_employed'].median()

In [ ]:
# Заполнение пропусков по доходу заемщика в зависимости от типа занятости
df.loc[(df['income_type']=='предприниматель')&(df['total_income'].isna()), 'total_income'] = median_income['median'][0]
df.loc[(df['income_type']=='компаньон')&(df['total_income'].isna()), 'total_income'] = median_income['median'][1]
df.loc[(df['income_type']=='госслужащий')&(df['total_income'].isna()), 'total_income'] = median_income['median'][2]
df.loc[(df['income_type']=='сотрудник')&(df['total_income'].isna()), 'total_income'] = median_income['median'][3]
df.loc[(df['income_type']=='безработный')&(df['total_income'].isna()), 'total_income'] = median_income['median'][4]
df.loc[(df['income_type']=='пенсионер')&(df['total_income'].isna()), 'total_income'] = median_income['median'][5]
df.loc[(df['income_type']=='студент')&(df['total_income'].isna()), 'total_income'] = median_income['median'][6]
df.loc[(df['income_type']=='в декрете')&(df['total_income'].isna()), 'total_income'] = median_income['median'][7]

In [ ]:
# Заполнение пропусков по трудовому стажу на медиану
df['days_employed'] = df['days_employed'].fillna(median_days_employed)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null float64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null float64
purpose             21454 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Прпоуски заполены, наши данные готовы для исследования

## Лемматизация

In [ ]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Все цели получения кредита можно разделить на 4 категории:  
1. Недвижимость
2. Образование
3. Автомобиль
4. Сввадьба


Проведем анализ - выявим цели кредита, которые встречаются чаще всего.  
Результат используем в категоризации.

In [ ]:
# Пропустим данные через цикл для лемматизации целей кредита и определения частот значений.
list_purpose_lemma = []
for purpose in df['purpose']:
    lemma = m.lemmatize(purpose)
    list_purpose_lemma.extend(lemma)
 
print(Counter(list_purpose_lemma))

Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


In [ ]:
df['purpose_lemma'] = df['purpose'].apply(m.lemmatize) # Применение метода лемматизации к целям кредита

# Функция присвоения категории по параметру - цель кредита.

def category_purpose (purpose_lemma):
    if ('жилье' in purpose_lemma) or ('недвижимость' in purpose_lemma):
        return 'Недвижимость'
    if 'образование' in purpose_lemma:
        return 'Образование'
    if 'автомобиль' in purpose_lemma:
        return 'Автомобиль'
    if 'свадьба'in purpose_lemma:
        return 'Свадьба'
    return 'Другое'

df['category_purpose'] = df['purpose_lemma'].apply(category_purpose) 




**Вывод**
***
Мы провели лемматизацию целей кредита и разделили данные на 4 категории.  
Это позволит нам посчитать количество заемщиков в каждой группе и решишить задачу о связи с возвратом кредита в срок

## Категоризация

Справочник: "Образование"

In [ ]:
directory_education = df.loc[:, ['education_id', 'education']]
directory_education = directory_education.drop_duplicates().reset_index(drop=True)
directory_education

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Справочник: "Семейный статус"

In [ ]:
directory_family = df.loc[:, ['family_status_id', 'family_status']]
directory_family = directory_family.drop_duplicates().reset_index(drop=True)
directory_family 

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


**Вывод**
***
Во входном датасете, данные по семейному статусу и образованию представлены вместе с id, что дает возможность вынести данные в отдельные справочники. Это даст нам определенные преимущества при индексации, так как:  

в поле *education* есть повторяющиеся слова - "высшее" и "неоконченное высшее"  
в поле *family_status* значения разделены через слеш.  


## Решение задачи

Проверим распределение по количеству заемщиков, которые не имели задолженностей по кредиту - 0, и тех, которые имели - 1

In [ ]:
df['debt'].value_counts()

0    19713
1     1741
Name: debt, dtype: int64

Наибольшее количество заемщиков оказались добросовестными.

### Определение зависимости между количество детей и возвратом кредита в срок

In [ ]:
df.groupby(['children'])['debt'].agg(['count','sum','mean']).sort_values(by='mean', ascending = False)


,count,sum,mean
children,,,
20,76,8,0.105263
4,41,4,0.097561
2,2052,194,0.094542
1,4855,445,0.091658
3,330,27,0.081818
0,14091,1063,0.075438
5,9,0,0.000000


**Вывод**
***
Исследование показало:  
чем больше детей в семье, тем больше доля не возврата кредиа в срок.  

В зоне риска оказываются заемщики, у котороых 20 детей, по ним процент не возврата кредита составил - 10 %  
В группах, где количество детей от 1 до 4, процент не возврата кредита - 9%  
Наименее рискованные заемщики без детей, процент не возврата по ним составил - 7%


### Определение зависимости между семейным положеним и возвратом кредита в срок

In [ ]:
pd.pivot_table(df, values=['debt'], index= ['family_status'], aggfunc='mean').sort_values(by='debt', ascending=False)

,debt
family_status,
Не женат / не замужем,0.097509
гражданский брак,0.093471
женат / замужем,0.075452
в разводе,0.071130
вдовец / вдова,0.065693


**Вывод**
***
Есть определенная зависимость возврата кредита в срок от семейного положения. 
Условно их можно разделить на группы:  
Замещики, не состоящие в браке имеют самый большой процент не возврата кредита - 10%.   
Процент не возврата по одиноким заемщикам, к ним мы относим категории вдовец/вдова и в разводе, составил 7%.  
Середину занимают заемщики в браке.

### Определение зависимости между доходом и возвратом кредита в срок

**Расчет влияния на основе данных с удаленными пропусками**

Определим группы для категоризации дохода.  
Будем полагаться на статистику, а именно - определим квартили наших данных

In [ ]:
statistics_notna = df_notna.describe()

In [ ]:
one_quartile_notna = int(statistics_notna['total_income']['25%'])
two_quartile_notna = int(statistics_notna['total_income']['50%'])
three_quartile_notna = int(statistics_notna['total_income']['75%'])

print('Первый квартиль: ', one_quartile_notna)
print('Второй квартиль: ', two_quartile_notna)
print('Третий квартиль: ', three_quartile_notna)

Первый квартиль:  103053
Второй квартиль:  145017
Третий квартиль:  203435


In [ ]:
# Функция присвоения категории по параметру - доход заемщика.

def category_income_notna (income):
    if income < one_quartile_notna:
        return 'Низкий'
    if one_quartile_notna <= income < two_quartile_notna:
        return 'Ниже среднего'
    if two_quartile_notna <= income < three_quartile_notna:
        return 'Выше среднего'
    if income >= three_quartile_notna:
        return 'Высокий'
    return 'Не определен'

In [ ]:
# Применим функцию для категоризации данных
df_notna['category_income'] = df_notna['total_income'].apply(category_income_notna)

In [ ]:
df_notna.groupby(['category_income'])['debt'].agg(['count','sum','mean']).sort_values(by='mean', ascending = False)

,count,sum,mean
category_income,,,
Выше среднего,4838,426,0.088053
Ниже среднего,4837,421,0.087037
Низкий,4838,383,0.079165
Высокий,4838,341,0.070484


**Вывод**
***
Явной зависимости не выявлено, но по доли не возврата кредита в срок можно сделать вывод, что заемщики с уровнем дохода от низкого до среднего имеют наибольшую долю - от 8% до 9%.  
Заемщики с высоким уровнем дохода можно отнести к доверительным, их доля составила - 7%.

**Расчет влияния на основе данных с заполненными пропусками**

Применим такую же методологию, только заменим данные

In [ ]:
statistics = df.describe()

In [ ]:
one_quartile = int(statistics['total_income']['25%'])
two_quartile = int(statistics['total_income']['50%'])
three_quartile = int(statistics['total_income']['75%'])

print('Первый квартиль: ', one_quartile)
print('Второй квартиль: ', two_quartile)
print('Третий квартиль: ', three_quartile)

Первый квартиль:  107623
Второй квартиль:  142594
Третий квартиль:  195820


In [ ]:
# Функция присвоения категории по параметру - доход заемщика.

def category_income (income):
    if income < one_quartile:
        return 'Низкий'
    if one_quartile <= income < two_quartile:
        return 'Ниже среднего'
    if two_quartile <= income < three_quartile:
        return 'Выше среднего'
    if income >= three_quartile:
        return 'Высокий'
    return 'Не определен'

In [ ]:
# Применим функцию для категоризации данных
df['category_income'] = df['total_income'].apply(category_income)

In [ ]:
df.groupby(['category_income'])['debt'].agg(['count','sum','mean']).sort_values(by='mean', ascending = False)

,count,sum,mean
category_income,,,
Ниже среднего,4409,385,0.087321
Выше среднего,6317,546,0.086433
Низкий,5364,427,0.079605
Высокий,5364,383,0.071402


**Вывод**
***
По сравнению с 1 варинатом расчета, произошли незначительные изменения в процентах не возврата кредита:  
По отношению к группе со средним доходом произошло снижение на 0,1 п.п.  
В группе с высоким доходом наоборт увеличение на 0,1 п.п.  
Группа с низким доходом осталась без изменений.  

Из этого можно сделать вывод, что пропуски были допущены преимущественно в категориях дохода выше нижнего уровня, т.е. больше 107623

#### Как разные цели кредита влияют на его возврат в срок

In [ ]:
df['category_purpose'].value_counts()

Недвижимость    10811
Автомобиль       4306
Образование      4013
Свадьба          2324
Name: category_purpose, dtype: int64

In [ ]:
pd.crosstab(df['category_purpose'], df['debt']).apply(lambda x: x/sum(x), axis=1).sort_values(by = [1], ascending = False)

debt,0,1
category_purpose,,
Автомобиль,0.906410,0.093590
Образование,0.907800,0.092200
Свадьба,0.919966,0.080034
Недвижимость,0.927666,0.072334


**Вывод**
***
Больше всего кредитов приходится на покупку недвижимости.  
Несмотря на это, количество случаев задолженности по этой группе самое низкое - 7%.  
Предположительно, это связано с самим залогом - недвижимость, из-за этого у заемщика ответственность становится выше.
Неблагоприятная ситуация с кредитами на покупку автомобиля или образование, процент не возврата кредита составил - 9%.

### Вывод
***
По результатам анализа мы выявили:  
1. Что с увеличением количества детей увеличивается риск не возврата кредита.   
   В зоне риска находятся заемщики, с количеством детей =20, по ним процент не возрата составил 10%.  
   Благонадежными можно считать заемщиков без детей, их процент не возврата составил -  7%
2. Заемщики, находящиеся в семейном статусе "вдовец/вдова" или "в разводе" наиболее благоприятная категория для одобрения кредита. Согласно данным, доля случаев задолженности составила - 7%. Не состоящие в браке или живущие в гражданском браке наименее благонадежные.
3. Явной зависимости дохода заемщика с возвратом кредита в срок не выявлено. 
4. Больше всего кредитов берут на покупку недвижимости, но в тоже врея эта категория имеет наименьший процент не возврата кредита - 7%. Стоит обратить внимание на потребности в получении кредита на покупку автомобиля или образования. По ним самые частые случаи не возврата кредита - 9%